**Лабораторная работа №3. Фильтрация изображений и
морфологические операции**

In [4]:
from PIL import Image, ImageDraw
import numpy as np
import random
import math

1) отфильтрованное монохромное (полутоновое) изображение


In [5]:
def monochrome(input_array):
  height, width = input_array.shape[:2]
  output_array = np.zeros((height, width), dtype=input_array.dtype)

  for y in range(height):
      for x in range(width):
          output_array[y][x] = np.mean(input_array[y][x])

  #return pim.fromarray(output_array, mode='L')
  new_image = Image.fromarray(output_array)
  new_image.save('new_image_monochrome.png')


2) разностное изображение (монохромный xor или модуль разности для полутона), операцией морфологического закрытия Closing

In [54]:
def salt_and_pepper(input_array, amount):
  height, width = input_array.shape[:2]
  output_array = input_array.copy()

  # Считаем количество точек для "соли" и "перца"
  num_salt = np.ceil(amount * height * width)
  num_pepper  = np.ceil(amount * height * width)

  coords_salt = [np.random.randint(0, i - 1, int(num_salt)) for i in [height, width]]
  output_array[coords_salt[0], coords_salt[1]] = 0


  coords_pepper = [np.random.randint(0, i - 1, int(num_pepper)) for i in input_array.shape]
  output_array[coords_pepper[0], coords_pepper[1]] = 255

  new_image = Image.fromarray(output_array)
  new_image.save('new_image_salt_and_pepper.png')

In [44]:
def erosion(input_array, kernel):
  height, width = input_array.shape[:2]
  kernel_height, kernel_width = kernel.shape

  eroded_image = np.zeros((height, width), dtype=np.uint8)
  pad_height, pad_width = kernel_height // 2, kernel_width // 2
  min_value = 255

  for i in range(pad_height, height - pad_height):
    for j in range(pad_width, width - pad_width):
      # Рассматриваем область входного изображения под ядром
      region = input_array[i-pad_height:i+pad_height+1, j-pad_width:j+pad_width+1]
      # Применяем операцию эрозии: находим минимум в области, где ядро == 1
      eroded_image[i, j] = np.min(region)


  return eroded_image

def dilate(input_array, kernel):
  height, width = input_array.shape[:2]
  kernel_height, kernel_width = kernel.shape

  dilate_image = np.zeros((height, width), dtype=np.uint8)
  pad_height, pad_width = kernel_height // 2, kernel_width // 2

  for i in range(pad_height, height - pad_height):
      for j in range(pad_width, width - pad_width):
        # Рассматриваем область входного изображения под ядром
        region = input_array[i-pad_height:i+pad_height+1, j-pad_width:j+pad_width+1]
        # Применяем операцию расширения: находим максимум в области, где ядро == 1
        dilate_image[i][j] = np.max(region)


  return dilate_image


def closing(input_array, kernel):
  temp = dilate(input_array, kernel)
  output = erosion(temp, kernel)

  output_image = Image.fromarray(output)
  output_image.save('new_image_closing.png')

In [55]:
def main():
  input_image = Image.open('девушка.png')
  input_array = np.array(input_image)
  #window_size = 15

  monochrome(input_array)
  amount = 0.05

  input_image = Image.open('new_image_monochrome.png')
  input_array = np.array(input_image)

  salt_and_pepper(input_array, amount)

  input_image = Image.open('new_image_salt_and_pepper.png')
  input_array = np.array(input_image)

  kernel = np.array([[1, 1, 1],
                     [1, 1, 1],
                     [1, 1, 1]], dtype=np.uint8)  # Определение ядра


  closing(input_array, kernel)

if __name__ == '__main__':
    main()